# ☕ Café Sales Data Cleaning

Welcome to my notebook!  

This project is about **cleaning a messy café sales dataset**.  
The dataset contains multiple issues such as:  

- Missing values in numeric and categorical columns  
- Wrong or placeholder values like `ERROR` or `UNKNOWN`  
- Inconsistent transaction dates  
- Plausibility errors (e.g., Quantity × Price not matching Total Spent)  

### Goals:
1. Load and inspect the raw dataset  
2. Identify missing values and inconsistencies  
3. Apply systematic **data cleaning steps**  
4. Recalculate values when possible  
5. Export a clean dataset and a list of invalid rows  
6. Summarize fixes in a **cleaning report**

In [1]:
import pandas as pd

# show 100 rows when printing a dataframe
pd.set_option('display.max_rows', 100)

# show all columns when printing a dataframe
pd.set_option('display.max_columns', None)

# do not truncate column content
pd.set_option('display.expand_frame_repr', False)

## 📥 Load the Dataset

In [2]:
# reading the csv file
try:
    df = pd.read_csv('/kaggle/input/cafe-sales-dirty-data-for-cleaning-training/dirty_cafe_sales.csv')
    print("file successfully read")
except FileNotFoundError:
    print("Error: file 'dirty_cafe_sales.csv' not found.")
    df = None
except pd.errors.EmptyDataError:
    print("Error: file is empty.")
    df = None
except pd.errors.ParserError:
    print("Error: file could not be read (parse error).")
    df = None
except Exception as e:
    print(f"Un unknown error occured: {e}")
    df = None

file successfully read


## 👀 First Look at the Data

In [3]:
# getting some information and the first 5 rows

display(df.head())
display(df.info())
display(df.describe())

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


None

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_1961373,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


## 🎲 Random Sample

In [4]:
# diving more into the csv
display(df.sample(100))

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
7385,TXN_9616828,Coffee,2,2.0,4.0,Digital Wallet,Takeaway,2023-12-20
5562,TXN_2855149,Sandwich,2,4.0,8.0,Cash,In-store,2023-07-16
3239,TXN_6345691,NaN,5,4.0,20.0,Credit Card,NaN,2023-09-03
2475,TXN_8774692,Tea,5,1.5,7.5,Digital Wallet,NaN,2023-07-12
1293,TXN_5301689,Cake,5,3.0,15.0,NaN,In-store,2023-04-11
5659,TXN_6266017,Cookie,4,1.0,4.0,UNKNOWN,In-store,2023-09-28
1606,TXN_5580933,Smoothie,3,4.0,NaN,Cash,Takeaway,2023-10-19
2991,TXN_4801617,Juice,5,3.0,15.0,NaN,NaN,2023-04-02
9446,TXN_4374260,Cookie,1,1.0,NaN,Cash,UNKNOWN,2023-04-20
2797,TXN_2936975,Tea,1,1.5,1.5,Digital Wallet,Takeaway,2023-10-05


## 🔍 Missing Values

In [5]:
# counting missing values in each column
missing_values_count = df.isnull().sum()

missing_values_count = df.isnull().sum()
display(missing_values_count)
display(df[df.isna().any(axis=1)])

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31
8,TXN_4717867,NaN,5,3.0,15.0,NaN,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,NaN,In-store,2023-12-31
13,TXN_9437049,Cookie,5,1.0,5.0,NaN,Takeaway,2023-06-01
14,TXN_8915701,ERROR,2,1.5,3.0,NaN,In-store,2023-03-21
...,...,...,...,...,...,...,...,...
9994,TXN_7851634,UNKNOWN,4,4.0,16.0,NaN,NaN,2023-01-08
9995,TXN_7672686,Coffee,2,2.0,4.0,NaN,UNKNOWN,2023-08-30
9996,TXN_9659401,NaN,3,NaN,3.0,Digital Wallet,NaN,2023-06-02
9997,TXN_5255387,Coffee,4,2.0,8.0,Digital Wallet,NaN,2023-03-02


## 🧹 Data Cleaning Steps

In [6]:
# Convert numeric columns to float

num_cols = ["Quantity", "Price Per Unit", "Total Spent"]
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Replace NaN in numeric columns with 0.0
num_cols = ["Quantity", "Price Per Unit", "Total Spent"]
df[num_cols] = df[num_cols].fillna(0.0)

### 🔢 Recalculating Numeric Fields

In [7]:
# Prepare counters for reporting
fixes = {"Price Per Unit": 0, "Total Spent": 0, "Quantity": 0, 
         "Item": 0, "Payment Method": 0, "Location": 0, "Transaction Date": 0}

# Recalculate missing Price Per Unit
mask_price_missing = (df["Price Per Unit"] == 0.0) & (df["Total Spent"] > 0) & (df["Quantity"] > 0)
fixes["Price Per Unit"] = mask_price_missing.sum()
df.loc[mask_price_missing, "Price Per Unit"] = df["Total Spent"] / df["Quantity"]

# Recalculate missing Total Spent
mask_total_missing = (df["Total Spent"] == 0.0) & (df["Price Per Unit"] > 0) & (df["Quantity"] > 0)
fixes["Total Spent"] = mask_total_missing.sum()
df.loc[mask_total_missing, "Total Spent"] = df["Price Per Unit"] * df["Quantity"]

# Recalculate missing Quantity
mask_quantity_missing = (df["Quantity"] == 0.0) & (df["Price Per Unit"] > 0) & (df["Total Spent"] > 0)
fixes["Quantity"] = mask_quantity_missing.sum()
df.loc[mask_quantity_missing, "Quantity"] = df["Total Spent"] / df["Price Per Unit"]

### 🧾 Cleaning Categorical Values

In [8]:
# Clean Item values
before_items = df["Item"].isin(["ERROR", "UNKNOWN"]).sum() + df["Item"].isna().sum()

df["Item"] = df["Item"].replace(["ERROR", "UNKNOWN"], pd.NA).fillna("Unknown Item")

n_salad = ((df["Price Per Unit"] == 5) & (df["Item"].str.lower() == "unknown item")).sum()
n_coffee = ((df["Price Per Unit"] == 2) & (df["Item"].str.lower() == "unknown item")).sum()
n_tea = ((df["Price Per Unit"] == 1.5) & (df["Item"].str.lower() == "unknown item")).sum()

df.loc[(df["Price Per Unit"] == 5) & (df["Item"].str.lower() == "unknown item"), "Item"] = "Salad"
df.loc[(df["Price Per Unit"] == 2) & (df["Item"].str.lower() == "unknown item"), "Item"] = "Coffee"
df.loc[(df["Price Per Unit"] == 1.5) & (df["Item"].str.lower() == "unknown item"), "Item"] = "Tea"

fixes["Item"] = before_items + n_salad + n_coffee + n_tea

# Clean Payment Method values
before_payment = df["Payment Method"].isin(["ERROR", "UNKNOWN"]).sum() + df["Payment Method"].isna().sum()
fixes["Payment Method"] = before_payment
df["Payment Method"] = df["Payment Method"].replace(["ERROR", "UNKNOWN"], pd.NA).fillna("Unknown")

# Clean Location values
before_location = df["Location"].isin(["ERROR", "UNKNOWN"]).sum() + df["Location"].isna().sum()
fixes["Location"] = before_location
df["Location"] = df["Location"].replace(["ERROR", "UNKNOWN"], pd.NA).fillna("Unknown")

# Clean Transaction Date values
before_date = df["Transaction Date"].isna().sum()
fixes["Transaction Date"] = before_date
df["Transaction Date"] = df["Transaction Date"].replace(["ERROR", "UNKNOWN"], pd.NA)
df["Transaction Date"] = pd.to_datetime(df["Transaction Date"], errors="coerce")

# Sort by Transaction Date (NaT will be sorted last)
df_sorted = df.sort_values(by="Transaction Date")
df = df_sorted

### ✅ Plausibility Check

In [9]:
# Plausibility check
df["Check_Total_OK"] = (df["Quantity"] * df["Price Per Unit"]).round(2) == df["Total Spent"].round(2)
invalid_rows = df[~df["Check_Total_OK"]]
invalid_count = len(invalid_rows)

# Save cleaned and invalid rows
df.to_csv("clean_cafe_sales.csv", index=False)
invalid_rows.to_csv("transactions_invalid.csv", index=False)

### 💾 Save & Export Results

In [10]:
# Save cleaned and invalid rows
df.to_csv("clean_cafe_sales.csv", index=False)
invalid_rows.to_csv("transactions_invalid.csv", index=False)

## 📋 Cleaning Report

In [11]:
for key, val in fixes.items():
    print(f"{key} fixed/filled: {val}")
print(f"Invalid rows (plausibility check failed): {invalid_count}")
print("Files saved: clean_cafe_sales.csv, transactions_invalid.csv")

Price Per Unit fixed/filled: 495
Total Spent fixed/filled: 462
Quantity fixed/filled: 441
Item fixed/filled: 1337
Payment Method fixed/filled: 3178
Location fixed/filled: 3961
Transaction Date fixed/filled: 159
Invalid rows (plausibility check failed): 18
Files saved: clean_cafe_sales.csv, transactions_invalid.csv


## 🔎 Inspect Cleaned Data

In [12]:
# data inspection
df2 = pd.read_csv("clean_cafe_sales.csv")
display(df2.sample(100))
display(df2.head(100))
display(df2.iloc[9450:9461])

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date,Check_Total_OK
9167,TXN_2161372,Juice,1.0,3.0,3.0,Credit Card,Takeaway,2023-12-17,True
3945,TXN_7423281,Juice,1.0,3.0,3.0,Digital Wallet,Unknown,2023-06-01,True
8471,TXN_5324330,Salad,3.0,5.0,15.0,Digital Wallet,Takeaway,2023-11-20,True
2602,TXN_2644962,Smoothie,2.0,4.0,8.0,Unknown,Unknown,2023-04-09,True
3980,TXN_4502314,Cookie,3.0,1.0,3.0,Credit Card,Unknown,2023-06-02,True
2130,TXN_5230719,Cake,5.0,3.0,15.0,Credit Card,Unknown,2023-03-23,True
6604,TXN_7714902,Coffee,4.0,2.0,8.0,Cash,Unknown,2023-09-10,True
3271,TXN_6352917,Salad,5.0,5.0,25.0,Cash,Unknown,2023-05-05,True
7433,TXN_3889213,Cake,1.0,3.0,3.0,Credit Card,Unknown,2023-10-13,True
7399,TXN_2258091,Juice,2.0,3.0,6.0,Unknown,In-store,2023-10-11,True


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date,Check_Total_OK
0,TXN_4801947,Juice,1.0,3.0,3.0,Digital Wallet,Takeaway,2023-01-01,True
1,TXN_9161256,Smoothie,2.0,4.0,8.0,Digital Wallet,In-store,2023-01-01,True
2,TXN_6093955,Tea,5.0,1.5,7.5,Unknown,Takeaway,2023-01-01,True
3,TXN_8842223,Sandwich,5.0,4.0,20.0,Digital Wallet,In-store,2023-01-01,True
4,TXN_7367474,Juice,5.0,3.0,15.0,Digital Wallet,Takeaway,2023-01-01,True
5,TXN_2690222,Unknown Item,4.0,3.0,12.0,Unknown,Unknown,2023-01-01,True
6,TXN_5728991,Salad,2.0,5.0,10.0,Unknown,Unknown,2023-01-01,True
7,TXN_2192787,Sandwich,5.0,4.0,20.0,Cash,In-store,2023-01-01,True
8,TXN_5563675,Tea,3.0,1.5,4.5,Digital Wallet,In-store,2023-01-01,True
9,TXN_2104473,Cake,3.0,3.0,9.0,Digital Wallet,Takeaway,2023-01-01,True


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date,Check_Total_OK
9450,TXN_1328953,Smoothie,3.0,4.0,12.0,Unknown,Unknown,2023-12-28,True
9451,TXN_2474395,Smoothie,4.0,4.0,16.0,Digital Wallet,Takeaway,2023-12-28,True
9452,TXN_3951329,Sandwich,1.0,4.0,4.0,Credit Card,Takeaway,2023-12-28,True
9453,TXN_9565246,Coffee,5.0,2.0,10.0,Cash,Unknown,2023-12-28,True
9454,TXN_4358673,Cookie,5.0,1.0,5.0,Digital Wallet,Takeaway,2023-12-28,True
9455,TXN_9096052,Smoothie,4.0,4.0,16.0,Credit Card,Takeaway,2023-12-28,True
9456,TXN_8091357,Coffee,1.0,2.0,2.0,Cash,Unknown,2023-12-28,True
9457,TXN_7728415,Tea,5.0,1.5,7.5,Credit Card,In-store,2023-12-28,True
9458,TXN_9834415,Cookie,2.0,1.0,2.0,Credit Card,In-store,2023-12-28,True
9459,TXN_3882139,Smoothie,4.0,4.0,16.0,Unknown,Unknown,2023-12-28,True


# 📊 Summary

### What we achieved:
- Cleaned and converted numeric columns (`Quantity`, `Price Per Unit`, `Total Spent`)  
- Replaced or inferred missing values  
- Fixed invalid entries in `Item`, `Payment Method`, `Location`, and `Transaction Date`  
- Sorted dataset chronologically  
- Added a plausibility check  
- Exported:
  - `clean_cafe_sales.csv` → cleaned dataset  
  - `transactions_invalid.csv` → invalid rows  
